In [7]:
%pip install pandas langchain-experimental matplotlib langchain-community langchain-ollama pypdf

Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain_ollama import OllamaEmbeddings

# Путь к файлу (мы в папке notebooks, поэтому выходим на уровень вверх .., потом в data/docs)
PDF_PATH = "../data/docs/sample.pdf"

# Загружаем
print("Загружаю PDF...")
loader = PyPDFLoader(PDF_PATH)
docs = loader.load()

# Склеиваем весь текст в одну большую строку для чистоты эксперимента
full_text = " ".join([d.page_content for d in docs])

print(f"Готово! Длина текста: {len(full_text)} символов.")

Загружаю PDF...
Готово! Длина текста: 13549 символов.


In [9]:
print("--- 1. FIXED SIZE CHUNKING ---")

# Настройки: 1000 символов, 0 перекрытия
splitter_fixed = CharacterTextSplitter(
    separator="", # Режем где попало, даже посреди слова
    chunk_size=1000,
    chunk_overlap=0
)
chunks_fixed = splitter_fixed.split_text(full_text)

print(f"Количество чанков: {len(chunks_fixed)}")

# Посмотрим, как он разрезал (ищем обрывки слов)
print(f"\nПример стыка (конец 1-го и начало 2-го чанка):")
print(f"ЧАНК 1 (конец): ...{chunks_fixed[0][-50:]}")
print(f"ЧАНК 2 (начало): {chunks_fixed[1][:50]}...")

--- 1. FIXED SIZE CHUNKING ---
Количество чанков: 14

Пример стыка (конец 1-го и начало 2-го чанка):
ЧАНК 1 (конец): ...-vector .svg 1 / 1 ,
S.G. CHEKANO V
Communic ate d
ЧАНК 2 (начало): by S.V. Sudopla to v
Abstract: The w ork has b egu...


In [10]:
print("--- 2. RECURSIVE CHUNKING ---")

# Настройки: 1000 символов, но с перекрытием 200 (чтобы не терять контекст на границах)
splitter_recursive = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""] # Приоритеты разделения
)
chunks_recursive = splitter_recursive.split_text(full_text)

print(f"Количество чанков: {len(chunks_recursive)}")
print(f"Первый чанк:\n{chunks_recursive[0][:200]}...")

--- 2. RECURSIVE CHUNKING ---
Количество чанков: 17
Первый чанк:
Math-Net.Ru
Общероссийский математический портал
A. A. Stepanova, E. L. Efremov, S. G. Chekanov, Pseudoﬁnite S-acts,
Сиб. электрон. матем. изв., 2024, том 21, выпуск 1, 271–276
https://www.mathnet.ru/...


In [11]:
print("--- 3. SEMANTIC CHUNKING ---")
print("Запускаю анализ смысла (может занять 1-2 минуты, так как работает локально)...")

# Подключаем локальные эмбеддинги
embeddings = OllamaEmbeddings(model="all-minilm")

# Создаем "умный" сплиттер
splitter_semantic = SemanticChunker(embeddings, breakpoint_threshold_type="percentile")

chunks_semantic = splitter_semantic.split_text(full_text)

print(f"Готово! Количество чанков: {len(chunks_semantic)}")

--- 3. SEMANTIC CHUNKING ---
Запускаю анализ смысла (может занять 1-2 минуты, так как работает локально)...
Готово! Количество чанков: 25


In [12]:
# Собираем статистику
data = {
    "Метод": ["Fixed (Тупой)", "Recursive (Стандарт)", "Semantic (Умный)"],
    "Кол-во чанков": [len(chunks_fixed), len(chunks_recursive), len(chunks_semantic)],
    "Средняя длина (символов)": [
        int(sum(len(c) for c in chunks_fixed)/len(chunks_fixed)),
        int(sum(len(c) for c in chunks_recursive)/len(chunks_recursive)),
        int(sum(len(c) for c in chunks_semantic)/len(chunks_semantic))
    ]
}

df = pd.DataFrame(data)
display(df) # Или просто print(df)

,Метод,Кол-во чанков,Средняя длина (символов)
0,Fixed (Тупой),14,967
1,Recursive (Стандарт),17,944
2,Semantic (Умный),25,540


ВЫВОД:
Для моего документа (научная статья по математике) лучшей стратегией показал себя Semantic Chunking.

1. Fixed Chunking разрывает слова и формулы, что недопустимо в точных науках.
2. Recursive Chunking хорош, но он объединяет несколько разных определений в один блок из-за большого размера (1000).
3. Semantic Chunking создал больше всего фрагментов (25), но они меньше по размеру (540 символов). Это значит, что он выделил каждое определение и теорему в отдельный логический блок. Это даст наиболее точные ответы при поиске.